In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from mlxtend.frequent_patterns import apriori, association_rules


Bad key "axes.color_cycle" on line 15 in
/Users/abhisheksingla/.matplotlib/stylelib/my_custom_style.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key "axes.color_cycle" on line 15 in
/Users/abhisheksingla/.matplotlib/stylelib/my_new_custom_style.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
path = "/".join(os.getcwd().split('/')[:-1])

### Data Reduction

In [30]:
order_product_prior= pd.read_csv(path+'/data/order_products__prior.csv')
orders_df= pd.read_csv(path+'/data/orders.csv')
products_df = pd.read_csv(path+'/src/product_data.csv')

In [31]:
#taking sample of 50000 users out of 206209 users in order to reduce the dataset
sample_users = pd.Series(orders_df.user_id.unique()).sample(500, random_state=42)

#Filter the existing dataset based on the sample and storing it in new dataframe
new_orders_df = orders_df[orders_df.user_id.isin(sample_users)]

#Now, filtering prior table which has 32M records
sample_prior_orders =new_orders_df[new_orders_df['eval_set']=='prior'].order_id.unique()
prior_data = order_product_prior[order_product_prior.order_id.isin(sample_prior_orders)]

In [32]:
del sample_users, order_product_prior

In [33]:
#Checking the update dataset after filtering 
prior_data.shape

(70768, 4)

In [34]:
prior_data.head()

,order_id,product_id,add_to_cart_order,reordered
1399,151,46931,1,0
1400,151,26152,2,1
1401,151,48697,3,1
1402,151,30486,4,1
4020,425,33290,1,0


In [35]:
prior_product_ids =prior_data.product_id.unique()
products_df = products_df[products_df.product_id.isin(prior_product_ids)]

#### Storing sampled dataset

In [37]:
#Storing the updated dataset into new csv file
prior_data.to_csv(path+'/model/order_product_prior_data.csv')
new_orders_df.to_csv(path+'/model/orders_data.csv')
products_df.to_csv(path+'/model/product_data.csv')

In [38]:
del prior_data, new_orders_df, products_df

# Dataset Used For Modelling

Final files:
1. Orders
2. Order Product Prior
4. Product

In [39]:
prior_df= pd.read_csv(path+'/model/order_product_prior_data.csv')
orders_df= pd.read_csv(path+'/model/orders_data.csv')
products_df= pd.read_csv(path+'/model/product_data.csv')

In [41]:
prior_df.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered
0,1399,151,46931,1,0
1,1400,151,26152,2,1
2,1401,151,48697,3,1
3,1402,151,30486,4,1
4,4020,425,33290,1,0


In [42]:
orders_df.head()

,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,4333,795884,273,prior,1,1,16,NaN
1,4334,2578458,273,prior,2,6,16,5.0
2,4335,2024887,273,prior,3,0,13,1.0
3,4336,1787108,273,prior,4,6,15,30.0
4,4337,2755405,273,prior,5,0,18,8.0


In [43]:
products_df.head()

,Unnamed: 0,Unnamed: 0.1,product_id,product_name,aisle_id,department_id,department,aisle
0,0,0,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes
1,9,9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7,beverages,water seltzer sparkling water
2,11,11,12,Chocolate Fudge Layer Cake,119,1,frozen,frozen dessert
3,19,19,20,Pomegranate Cranberry & Aloe Vera Enrich Drink,98,7,beverages,juice nectars
4,24,24,25,Salted Caramel Lean Protein & Fiber Bar,3,19,snacks,energy granola bars


#### Mapping Product Info to Prior Data

In [7]:
prior_df=pd.merge(prior_df,products_df[['product_id','product_name','aisle_id','aisle']],how='left',left_on= 'product_id',right_on='product_id')

### Prior Dataset

In [8]:
prior_df.shape

(7927707, 8)

In [18]:
prior_df.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,aisle
0,3018,326,429,1,1,Dark Red Kidney Beans,59,canned meals beans
1,3019,326,35042,2,1,Black Beans,59,canned meals beans
2,3020,326,21010,3,1,Organic Crushed Tomatoes with Basil,81,canned jarred vegetables
3,3021,326,27985,4,1,Smartwater Electrolyte Enhanced Water,115,water seltzer sparkling water
4,3022,326,23219,5,1,Nutz Over Chocolate,3,energy granola bars


#### Top aisles:

In [19]:
top_aisles = prior_df.groupby('aisle_id')['order_id'].count().sort_values(ascending=False).head(40).index

filtering prior data based on top aisles

In [20]:
new_prior_df = prior_df[prior_df['aisle_id'].isin(top_aisles)]

#### Popular Products across each aisle

In [21]:
grouped_data = new_prior_df.groupby(['aisle_id','product_id'])['order_id'].count().reset_index()

In [22]:
grouped_data.rename(columns={'order_id': 'no_of_order'}, inplace=True)

In [23]:
grouped_data.head()

,aisle_id,product_id,no_of_order
0,3,25,11
1,3,594,1
2,3,1364,1
3,3,1426,1
4,3,1452,3


#### Top product
##### As we have 14k distinct product which is leading to large compuatation time so in order to speed up we're calculated Top 15 products across each aisle.

In [34]:
top_product = grouped_data.groupby(['aisle_id'])['no_of_order'].rank(ascending=False).apply(lambda x: True if x<=15 else False)

In [35]:
top_product.sum()

591

#### Joining sampled products with prior data

In [57]:
limited_prior = pd.merge(grouped_data[top_product].drop(['aisle_id','no_of_order'],axis=1), prior_df, how='inner', left_on='product_id',right_on='product_id')

##### Products

In [58]:
print("Number of products in old dataset: ",prior_df.product_id.nunique())

Number of products in old dataset:  10911


In [59]:
print("Number of products in new dataset: ",limited_prior.product_id.nunique())

Number of products in new dataset:  591


##### Orders

In [60]:
print("Number of orders in old dataset: ",prior_df.order_id.nunique())

Number of orders in old dataset:  8273


In [61]:
print("Number of oreders in new dataset: ",limited_prior.order_id.nunique())

Number of oreders in new dataset:  7381


In [62]:
sample_prior_df = limited_prior.copy()

#### Product and Order Data Matrix

In [63]:
basket = pd.crosstab(sample_prior_df['order_id'], sample_prior_df['product_name'])

In [64]:
basket.head()

product_name,100% Lemon Juice,100% Lime Juice,100% Pure Pumpkin,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat DHA Omega-3 Reduced Fat Milk,2% Reduced Fat Milk,2% Reduced Fat Organic Milk,70% Dark Chocolate Salted Almond Bar,Aged White Cheddar Baked Rice & Corn Puffs Gluten Free Lunch Packs,...,Whole Grain Oat Cereal,Whole Milk,Whole Wheat Bread,Whole Wheat English Muffins,Yellow Onions,"YoKids Squeezers Organic Low-Fat Yogurt, Strawberry","Yogurt, Lowfat, Strawberry",ZBar Organic Chocolate Brownie Energy Snack,Zero Calorie Sweetener,with Crispy Almonds Cereal
order_id,,,,,,,,,,,,,,,,,,,,,
326,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
636,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1045,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Apriori Algorithm

In [65]:
frq_items = apriori(basket, min_support = 0.01, use_colnames = True,max_len=3) 

In [66]:
frq_items

,support,itemsets
0,0.016935,(100% Raw Coconut Water)
1,0.015310,(100% Whole Wheat Bread)
2,0.036038,(Apple Honeycrisp Organic)
3,0.032787,(Asparagus)
4,0.134264,(Bag of Organic Bananas)
...,...,...
130,0.010839,"(Organic Baby Spinach, Organic Raspberries)"
131,0.015174,"(Organic Baby Spinach, Organic Strawberries)"
132,0.011381,"(Organic Hass Avocado, Organic Raspberries)"
133,0.015310,"(Organic Hass Avocado, Organic Strawberries)"


### Association Rules for multiple combinations of products

In [67]:
# Building the model 
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 2) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.sort_values(by='confidence',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
14,(Organic Peeled Whole Baby Carrots),(Banana),0.023574,0.193876,0.010026,0.425287,2.193603,0.005455,1.402655
7,(Organic Large Extra Fancy Fuji Apple),(Bag of Organic Bananas),0.029942,0.134264,0.011110,0.371041,2.763523,0.007090,1.376459
2,(Organic Cucumber),(Bag of Organic Bananas),0.028045,0.134264,0.010161,0.362319,2.698562,0.006396,1.357632
11,(Organic Unsweetened Almond Milk),(Bag of Organic Bananas),0.030077,0.134264,0.010703,0.355856,2.650426,0.006665,1.344010
4,(Organic Hass Avocado),(Bag of Organic Bananas),0.076954,0.134264,0.026555,0.345070,2.570096,0.016222,1.321877
1,(Apple Honeycrisp Organic),(Bag of Organic Bananas),0.036038,0.134264,0.012329,0.342105,2.548011,0.007490,1.315919
19,(Organic Baby Carrots),(Organic Baby Spinach),0.031703,0.101748,0.010297,0.324786,3.192074,0.007071,1.330323
8,(Organic Lemon),(Bag of Organic Bananas),0.035090,0.134264,0.010161,0.289575,2.156766,0.005450,1.218618
12,(Organic Yellow Onion),(Bag of Organic Bananas),0.038884,0.134264,0.011110,0.285714,2.128009,0.005889,1.212031
26,(Organic Raspberries),(Organic Strawberries),0.061780,0.090503,0.015039,0.243421,2.689657,0.009447,1.202118


## Collaborative Filtering

In [68]:
import scipy.sparse as sps
from sklearn.preprocessing import normalize

In [69]:
def Naive_CF(data,users):
    user_similarity_cosine = sps.csr_matrix((users, users))
    data_N = normalize(data, norm='l2', axis=1)
    user_similarity_cosine = np.dot(data_N,data_N.T)
    return user_similarity_cosine,np.dot(user_similarity_cosine,data)

In [72]:
#Dropping unwanted columns
limited_prior.drop('Unnamed: 0', axis=1, inplace=True)

#### Mapping UserId to Prior Dataset

In [70]:
limited_prior = pd.merge(limited_prior, orders_df[['order_id','user_id']], how='left', left_on='order_id', right_on='order_id')

In [73]:
limited_prior.head()

,product_id,order_id,add_to_cart_order,reordered,product_name,aisle_id,aisle,user_id
0,5322,187764,8,1,Gluten Free Dark Chocolate Chunk Chewy with a ...,3,energy granola bars,184501
1,5322,289567,19,1,Gluten Free Dark Chocolate Chunk Chewy with a ...,3,energy granola bars,142039
2,5322,326427,3,0,Gluten Free Dark Chocolate Chunk Chewy with a ...,3,energy granola bars,41700
3,5322,336524,6,1,Gluten Free Dark Chocolate Chunk Chewy with a ...,3,energy granola bars,184501
4,5322,814818,10,1,Gluten Free Dark Chocolate Chunk Chewy with a ...,3,energy granola bars,184501


### User - Product Data Matrix

In [74]:
data = limited_prior.groupby(['user_id','product_name'])['order_id'].count().unstack().fillna(0)

In [75]:
data.head()

product_name,100% Lemon Juice,100% Lime Juice,100% Pure Pumpkin,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat DHA Omega-3 Reduced Fat Milk,2% Reduced Fat Milk,2% Reduced Fat Organic Milk,70% Dark Chocolate Salted Almond Bar,Aged White Cheddar Baked Rice & Corn Puffs Gluten Free Lunch Packs,...,Whole Grain Oat Cereal,Whole Milk,Whole Wheat Bread,Whole Wheat English Muffins,Yellow Onions,"YoKids Squeezers Organic Low-Fat Yogurt, Strawberry","Yogurt, Lowfat, Strawberry",ZBar Organic Chocolate Brownie Energy Snack,Zero Calorie Sweetener,with Crispy Almonds Cereal
user_id,,,,,,,,,,,,,,,,,,,,,
112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2145,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Passing user-product matrix to the "Navie-CF (Collaborating Filtering)"

In [76]:
UserMatrix, CFresult = Naive_CF(data,orders_df.user_id.nunique())
CFresult

array([[0.48133263, 0.70933831, 1.50759715, ..., 2.33732956, 1.32962142,
        0.26031843],
       [0.71673189, 0.72993461, 1.67620138, ..., 1.51570064, 0.83331558,
        1.65031417],
       [0.56839495, 0.1346901 , 1.8164397 , ..., 1.20442241, 0.51599081,
        1.35947595],
       ...,
       [1.31959697, 1.36638298, 4.64047581, ..., 3.85789683, 1.68987068,
        0.26075676],
       [0.11376653, 1.26246914, 3.13341152, ..., 2.12039914, 0.44847167,
        0.        ],
       [1.67728254, 0.90597831, 2.23829391, ..., 3.88224252, 1.12416089,
        0.08984269]])

In [297]:
pd.Series(CFresult[0][(-1*CFresult[0]).argsort()]).describe()

count    591.000000
mean       5.005322
std       10.037864
min        0.000000
25%        1.061702
50%        2.300866
75%        4.691120
max      126.278899
dtype: float64

#### User-User Matrix

User-User matrix - Using this, we are finding the similar users based on collaborating filtering

In [81]:
pd.DataFrame(UserMatrix, index=data.T.columns, columns=data.T.columns)

user_id,112,701,736,1162,2145,2187,2316,3275,3512,4958,...,202219,202520,203245,203631,203753,203807,203986,204180,204221,205640
user_id,,,,,,,,,,,,,,,,,,,,,
112,1.000000,0.125243,0.000000,0.000000,0.000000,0.025710,0.211984,0.054750,0.061399,0.258572,...,0.088098,0.006574,0.028510,0.000000,0.000000,0.001673,0.0,0.097379,0.006730,0.112746
701,0.125243,1.000000,0.041523,0.011078,0.000000,0.000000,0.100315,0.226114,0.197221,0.138196,...,0.100056,0.099543,0.000000,0.075810,0.041523,0.086141,0.0,0.204177,0.081531,0.086976
736,0.000000,0.041523,1.000000,0.000000,0.000000,0.187523,0.000000,0.054456,0.101780,0.138675,...,0.000000,0.000000,0.181956,0.000000,0.000000,0.030508,0.0,0.000000,0.000000,0.000000
1162,0.000000,0.011078,0.000000,1.000000,0.057874,0.000000,0.017187,0.014528,0.023533,0.000000,...,0.021429,0.038374,0.000000,0.000000,0.013339,0.022790,0.0,0.029152,0.000000,0.034150
2145,0.000000,0.000000,0.000000,0.057874,1.000000,0.000000,0.026204,0.019688,0.103036,0.000000,...,0.000000,0.104010,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.021037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203807,0.001673,0.086141,0.030508,0.022790,0.000000,0.000000,0.130703,0.103003,0.072867,0.077845,...,0.151930,0.207712,0.000000,0.044560,0.140338,1.000000,0.0,0.236692,0.000000,0.178932
203986,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055012,0.000000,0.026487,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
204180,0.097379,0.204177,0.000000,0.029152,0.000000,0.000000,0.215591,0.277687,0.029658,0.090920,...,0.219423,0.301253,0.099413,0.000000,0.136590,0.236692,0.0,1.000000,0.120689,0.228887


#### Final User Item Matrix

In [82]:
UserItemMatrix = pd.DataFrame(CFresult, columns=data.columns, index=data.index)

In [83]:
UserItemMatrix.head()

product_name,100% Lemon Juice,100% Lime Juice,100% Pure Pumpkin,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat DHA Omega-3 Reduced Fat Milk,2% Reduced Fat Milk,2% Reduced Fat Organic Milk,70% Dark Chocolate Salted Almond Bar,Aged White Cheddar Baked Rice & Corn Puffs Gluten Free Lunch Packs,...,Whole Grain Oat Cereal,Whole Milk,Whole Wheat Bread,Whole Wheat English Muffins,Yellow Onions,"YoKids Squeezers Organic Low-Fat Yogurt, Strawberry","Yogurt, Lowfat, Strawberry",ZBar Organic Chocolate Brownie Energy Snack,Zero Calorie Sweetener,with Crispy Almonds Cereal
user_id,,,,,,,,,,,,,,,,,,,,,
112,0.481333,0.709338,1.507597,18.174872,7.383691,4.594874,4.314793,3.489026,1.352748,5.059380,...,0.980275,2.874477,3.946348,1.556142,13.076314,4.570555,4.519180,2.337330,1.329621,0.260318
701,0.716732,0.729935,1.676201,8.156555,8.955766,4.139229,3.843926,3.475671,1.000013,2.894613,...,1.864280,7.844922,3.499590,1.632818,22.495109,5.493759,3.053131,1.515701,0.833316,1.650314
736,0.568395,0.134690,1.816440,8.902059,5.077326,2.096423,0.403070,2.388033,1.990692,1.908121,...,0.875812,1.671375,5.532632,1.876001,9.104673,3.680774,7.744635,1.204422,0.515991,1.359476
1162,0.722001,0.223799,0.218845,0.216058,2.201329,0.641497,1.809846,1.236971,0.057971,0.264324,...,0.686211,3.852335,0.840510,0.185530,4.549678,1.278099,0.631702,0.329361,1.458457,0.975803
2145,2.180864,0.276259,0.530080,0.646962,2.103122,0.497865,3.282494,1.217040,0.137965,0.388793,...,0.290597,1.045626,0.450424,0.149346,2.959136,0.372966,0.105184,0.038997,0.179640,0.000000


#### Top products which may be ordered by a particular user

In [89]:
UserItemMatrix.iloc[0].sort_values(ascending=False).head(10)

product_name
Organic Hass Avocado      126.278899
Bag of Organic Bananas    105.040237
Organic Strawberries       75.263866
Banana                     75.181603
Organic Baby Spinach       75.072118
Organic Raspberries        52.010677
Organic Blueberries        36.547028
Organic Avocado            32.637648
Organic Yellow Onion       31.285809
Organic Grape Tomatoes     29.675243
Name: 112, dtype: float64

In [90]:
UserItem_df = UserItemMatrix.unstack().reset_index()

In [91]:
UserItem_df.head()

,product_name,user_id,0
0,100% Lemon Juice,112,0.481333
1,100% Lemon Juice,701,0.716732
2,100% Lemon Juice,736,0.568395
3,100% Lemon Juice,1162,0.722001
4,100% Lemon Juice,2145,2.180864


In [92]:
UserItem_df.rename(columns={0: 'likely'}, inplace=True)

In [93]:
UserItem_df.sort_values(by='user_id').head()

,product_name,user_id,likely
0,100% Lemon Juice,112,0.481333
114376,No Pulp Calcium & Vitamin D Pure Orange Juice,112,1.139043
15776,Beef Merlot,112,0.386583
246993,Simple Favorites Chicken Fettuccini Frozen Entree,112,0.051535
275587,Unsweetened Almondmilk,112,9.225620


#### Top 15 products for each user

In [94]:
top_r_prod = UserItem_df.groupby('user_id')['likely'].rank(ascending=False).apply(lambda x: True if x<=15 else False)

In [95]:
UserItem_df[top_r_prod].sort_values(by=['user_id', 'likely'],ascending=False).head(20)

,product_name,user_id,likely
14296,Banana,205640,363.883859
176000,Organic Raspberries,205640,118.032751
187832,Organic Strawberries,205640,91.202243
124728,Organic Baby Spinach,205640,87.341021
13310,Bag of Organic Bananas,205640,73.548533
123249,Organic Avocado,205640,70.818997
129658,Organic Blueberries,205640,57.781005
156280,Organic Hass Avocado,205640,54.238558
200650,Organic Whole Milk,205640,52.003244
95148,Large Lemon,205640,51.074803
